<a href="https://colab.research.google.com/github/TurkuNLP/Text_Mining_Course/blob/master/model_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/TurkuNLP/Text_Mining_Course/blob/master/train_bert_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<span style="color:red">**THIS NOTEBOOK IS NOT READY YET! THIS IS A BACKUP COPY!**</span>

# Train BERT text classifier using the Transformers library

This notebook shows how to train a simple text classifier by fine-tuning a pre-trained BERT model using the Hugging Face [Transformers](https://huggingface.co/transformers/) library.

This notebook is based in part on the [Text classification on GLUE](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb) notebook.

**NOTE**: it's recommended to run this using a runtime with a GPU. Select "Runtime" -> "Change runtime type" from the top menu in Colab and set "Hardware accelerator" to "GPU" when starting

## Install libraries

First, we'll use [`pip`](https://pypi.org/project/pip/) to install two Python libraries that are used in this notebook: [`transformers`](https://huggingface.co/transformers/) and [`datasets`](https://huggingface.co/docs/datasets/).

In [1]:
!pip --quiet install transformers
!pip --quiet install datasets

     |████████████████████████████████| 2.1MB 5.9MB/s 
     |████████████████████████████████| 3.3MB 55.3MB/s 
     |████████████████████████████████| 901kB 59.4MB/s 
     |████████████████████████████████| 204kB 5.9MB/s 
     |████████████████████████████████| 245kB 38.7MB/s 
     |████████████████████████████████| 112kB 62.2MB/s 


## Import libraries

Next, let's import the classes and functions we'll be using (click links for documentation):

* [AutoTokenizer](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoTokenizer): tokenizer class with support for the tokenization conventions of various pre-trained models
* [AutoModelForSequenceClassification](https://huggingface.co/transformers/model_doc/auto.html#automodelforsequenceclassification): an [AutoModel](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoModel) class that can load various pre-trained models and supports fine-tuning for text (sequence) classification
* [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments): simple class for holding hyperparameters and other settings for model training
* [Trainer](https://huggingface.co/transformers/main_classes/trainer.html): class supporting various forms of training transformer models
* [load_dataset](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset): function for loading datasets e.g. from the [datasets](https://huggingface.co/datasets) collection

In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_dataset

## Set model, dataset and hyperparameters

Let's then set some global variables such as the name of the pre-trained model and the hyperparameters we'll use for fine-tuning it.

* `MODEL_NAME`: the name of a pretrained model included in the [model repository](https://huggingface.co/models)
* `DATASET`: the path and name of a dataset included in the [dataset repository](https://huggingface.co/datasets)
* `LEARNING_RATE`, `BATCH_SIZE`, and `TRAIN_EPOCHS`: hyperparameters to use for fine-turning the model. (Try different values here!)

Here, we'll use the [Stanford Sentiment Treebank](https://huggingface.co/datasets/sst) dataset as prepared for the [GLUE](https://huggingface.co/datasets/glue) collection. This is a binary text classification task where the objective is to determine if sentences express a positive or negative sentiment.

In [5]:
MODEL_NAME = 'bert-base-cased'
DATASET = ('glue', 'sst2')

## Load dataset

We'll first load the dataset with [`load_dataset`](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset) and determine the number of distinct labels in the (training) data.

In [6]:
dataset = load_dataset(*DATASET)
num_labels = len(set(dataset['train']['label']))

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


The loaded dataset is a simple dictionary-like container for distinct `DataSet` objects for training, development (validation), and test data:

In [7]:
print(dataset)
print(f'number of distinct labels: {num_labels}')

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
number of distinct labels: 2


To reduce training time a bit, let's just take every 10th item in the `train` subset of the dataset.

In [8]:
dataset['train'] = dataset['train'].filter(lambda example, idx: idx % 10 == 0, with_indices=True)

## Load tokenizer and tokenize data

We'll then load an appropriate tokenizer for the pre-trained model we'll be using with [`AutoTokenizer.from_pretrained`](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoTokenizer.from_pretrained)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

The tokenizer can take a string (or e.g. a list of strings) and performs all the necessary preprocessing steps to prepare its input for use by the model, including splitting input texts into tokens, adding padding and special characters, and mapping those to integer IDs (`input_ids`). The tokenizer also creates attention weights (`attention_mask`) for the model.

In [10]:
tokenizer('Hello, BERT!')    # Demonstrate tokenizer output

{'input_ids': [101, 8667, 117, 139, 9637, 1942, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

We'll then define an encoding function applying the tokenizer to the text data  (key `"sentence"`) of a [`Dataset`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=dataset#datasets.Dataset) object, and use the `map` function of the `DatasetDict` to tokenize the train, development, and test datasets.

In [11]:
def encode_dataset(d):
  return tokenizer(d['sentence'])

encoded_dataset = dataset.map(encode_dataset)

## Load pre-trained model

Next, we'll load the pre-trained model with support for text classification output using [`AutoModelForSequenceClassification.from_prertained`](https://huggingface.co/transformers/model_doc/auto.html#transformers.AutoModelForSequenceClassification.from_pretrained).

Note that we need to provide the number of labels in the data when loading the model.

In [12]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

## Training parameters and metrics
We're almost ready to train. We'll next create a [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) object to hold the hyperparameters and other settings that are appropriate for training on Colab:

* `save_strategy`: set to `"no"` so that model checkpoints are not saved to avoids exhausting Colab storage space
* `evaluation_strategy` and `logging_strategy` set to `"epoch"` so that evaluation and logging are performed once per epoch
* The hyperparameters `LEARNING_RATE`, `BATCH_SIZE` and `TRAIN_EPOCHS` set above are passed to the training process through this object

Finally, we'll define a simple accuracy metric measuring how many predictions match their correct values.

In [13]:
def compute_accuracy(pred):
    y_pred = pred.predictions.argmax(axis=1)
    y_true = pred.label_ids
    return { 'accuracy': sum(y_pred == y_true) / len(y_true) }

# Training (fine-tuning)

For fine-tuning the pre-trained model, we'll create a [`Trainer`](https://huggingface.co/transformers/main_classes/trainer.html) object, providing it with the pre-trained model, settings, training and development (validation) data, and the evaluation metric created above.

In [14]:


# Hyperparameters
LEARNING_RATE=1e-4
BATCH_SIZE=128
TRAIN_EPOCHS=2

In [15]:
train_args = TrainingArguments(
    'output_dir',    # output directory for checkpoints and predictions
    save_strategy='no',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    num_train_epochs=TRAIN_EPOCHS,
)

In [16]:
trainer = Trainer(
      model,
      train_args,
      train_dataset=encoded_dataset['train'],
      eval_dataset=encoded_dataset['validation'],
      tokenizer=tokenizer,
      compute_metrics=compute_accuracy
)

Training is then performed simply by calling the `train` function of the `Trainer` object.

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.411100,0.259865,0.897936,3.021400,288.604000
2,0.144800,0.286207,0.888761,3.024700,288.290000


TrainOutput(global_step=106, training_loss=0.27793230200713537, metrics={'train_runtime': 79.6469, 'train_samples_per_second': 1.331, 'total_flos': 458337020915640.0, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 1712799744, 'init_mem_gpu_alloc_delta': 433776128, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 13778944, 'train_mem_gpu_alloc_delta': 1318304256, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 6710576128})

## Evaluation and predictions for user input

We can use `trainer.evaluate()` to evaluate the trained model on the `eval_dataset` given to the trainer:

In [18]:
results = trainer.evaluate()
print(f'Accuracy: {results["eval_accuracy"]}')

Accuracy: 0.8887614678899083


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
import torch
torch.save(model,"/content/drive/MyDrive/WorkStuff/sent_model.pt")

The trained model is available as `trainer.model`, and we can use it like any other model. Here we define a function to predict the sentiment a user-given string.

In [31]:
model = torch.load("/content/drive/MyDrive/WorkStuff/sent_model.pt")
model.to('cpu')    # simplifies input placement

sentiment_label = [
    'negative',
    'positive'
]

def predict_sentiment(string):
    tokenized = tokenizer(string, return_tensors='pt')
    pred = model(**tokenized)
    pred_idx = pred.logits.detach().numpy().argmax()
    return sentiment_label[pred_idx]

try that out

In [32]:
example_sentences = [
    'I think this is very good, and I am in support.',
    'This is not very good news.',
    'This could not have been better.',
    'This is not good news at all.',
    'Donald Trump lost the election and will no longer be every day on CNN.',
    'I think your service fulfilled my expectations, I could not really expect you to be any better'
]

for e in example_sentences:
    print(e, '->', predict_sentiment(e))

I think this is very good, and I am in support. -> positive
This is not very good news. -> negative
This could not have been better. -> negative
This is not good news at all. -> negative
Donald Trump lost the election and will no longer be every day on CNN. -> negative
I think your service fulfilled my expectations, I could not really expect you to be any better -> positive


Can you change the model, the hyperparameters, or some other aspect of the fine-tuning process to make the result better?

In [33]:
!pip install captum pandas matplotlib seaborn

In [34]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [35]:
model.eval()
model.zero_grad()

In [36]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    return model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )
    
def pos_forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None, position=0):
    pred = predict(inputs,
                   token_type_ids=token_type_ids,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    pred = pred[position]
    return pred.max(1).values

ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence


In [53]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id, device):
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id] 

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, device):
    seq_len = input_ids.size(1)
    token_type_ids = torch.zeros((1,seq_len), dtype=torch.long, device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids, device):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids,device):
    return torch.ones_like(input_ids,device=device)



In [54]:
device=model.device
input_ids, ref_input_ids, sep_id = construct_input_ref_pair("This piece of news is rather unfortunate, I wish it came out better", ref_token_id, sep_token_id, cls_token_id, device)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, device)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids, device)
attention_mask = construct_attention_mask(input_ids, device)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [56]:
p=predict(input_ids,token_type_ids=token_type_ids,position_ids=position_ids,attention_mask=attention_mask)

In [57]:
p

SequenceClassifierOutput([('logits',
                           tensor([[ 2.0345, -2.0113]], grad_fn=<AddmmBackward>))])

In [58]:
lig = LayerIntegratedGradients(pos_forward_func, model.bert.embeddings)

In [59]:
attributions_0, delta_0 = lig.attribute(inputs=input_ids,
                                  baselines=ref_input_ids,
                                  additional_forward_args=(token_type_ids, position_ids, attention_mask, 0),
                                  return_convergence_delta=True)

In [61]:
attributions_0.shape, delta_0.shape


(torch.Size([1, 16, 768]), torch.Size([1]))

In [63]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

attributions_0_sum = summarize_attributions(attributions_0)

In [69]:
print(attributions_0_sum)
print(tokenizer.convert_ids_to_tokens(input_ids[0]))

for a,t in zip(attributions_0_sum,tokenizer.convert_ids_to_tokens(input_ids[0])):
    print(float(a),t)

tensor([ 0.0000,  0.0507, -0.0797, -0.0169, -0.0172,  0.0897,  0.3466,  0.6010,
        -0.1844, -0.1429,  0.0211,  0.1008, -0.1773, -0.0258,  0.6356,  0.0000],
       dtype=torch.float64)
['[CLS]', 'This', 'piece', 'of', 'news', 'is', 'rather', 'unfortunate', ',', 'I', 'wish', 'it', 'came', 'out', 'better', '[SEP]']
0.0 [CLS]
0.05072412771318426 This
-0.07973131503258431 piece
-0.016872450022487197 of
-0.01723090692748148 news
0.0897154634569187 is
0.34657761694501293 rather
0.6010220697014054 unfortunate
-0.18437458399486942 ,
-0.14293074869932165 I
0.021141537443567577 wish
0.10077962073767636 it
-0.1772668128347815 came
-0.025793727070663516 out
0.6355945542022341 better
0.0 [SEP]
